In [1]:
%load_ext lab_black

In [4]:
from sadie.airr import Airr
import pandas as pd
from pathlib import Path

airr_api = Airr("human")
heavy_file = Path("../../tests/integration/airr/fixtures/catnap_nt_heavy.fasta.gz")
light_file = Path("../../tests/integration/airr/fixtures/catnap_nt_light.fasta.gz")
airr_api = Airr(species="human", database="imgt", functional="functional")
catnap_heavy = airr_api.run_file(heavy_file)
catnap_light = airr_api.run_file(light_file)
light_at = pd.read_feather(
    "../../tests/integration/airr/fixtures/catnap_light_airrtable.feather",
)
heavy_at = pd.read_feather(
    "../../tests/integration/airr/fixtures/catnap_heavy_airrtable.feather",
)
pd.testing.assert_frame_equal(light_at, catnap_light.table)
pd.testing.assert_frame_equal(heavy_at, catnap_heavy.table)

# make new files

In [3]:
catnap_heavy.table.to_feather(
    "../../tests/integration/airr/fixtures/catnap_heavy_airrtable.feather",
)

catnap_light.table.to_feather(
    "../../tests/integration/airr/fixtures/catnap_light_airrtable.feather",
)